In [10]:
#*******************************************************************************
#Function	  : writeToFile
#Description  : write unique reads to a file
#Parameters	  : read - unique read to be written to a file
#               file - output file
#Returned	  : none
#*******************************************************************************

def writeToFile(out, dictionary):
    for value in dictionary.values():
        out.write(str(value))
        out.write("\n")
        
#*******************************************************************************
#Function	  : adjustPosForward
#Description  : adjust the left most position taking into account soft clipping 
#Parameters	  : read - sam file read with information on soft clipping and position
#Returned	  : position - adjusted position of read based on CIGAR string
#*******************************************************************************
import re

def adjustPosForward(read):
    adjustment = 0
    unit = ""
    first = True
    read = read.strip()
    read = read.split("\t")
    position = int(read[3])
    cigar = read[5]
    for char in cigar:
        if str.isdigit(char): 
            unit = unit + char
        else:
            unit = unit + char
            #adjust for left S
            mMatch = re.match("([0-9]+)S", unit)
            if mMatch:
                adjust = mMatch.group(1)
                if first == True:
                    adjustment = adjustment - int(adjust)
            break
    return (position + adjustment)

#*******************************************************************************
#Function	  : adjustPosReverse
#Description  : adjust the left most position to create right most mapping position 
#              taking into account soft clipping 
#Parameters	  : read - sam file read with information on soft clipping and position
#Returned	  : position - adjusted position of read based on CIGAR string
#*******************************************************************************

def adjustPosReverse(read):
    adjustment = 0
    unit = ""
    first = True
    read = read.strip()
    read = read.split("\t")
    position = int(read[3])
    cigar = read[5]
    for char in cigar:
        if str.isdigit(char): 
            unit = unit + char
        else:
            unit = unit + char
            #adjust for left S IGNORE
            #if first:
                #adjustment = adjustment #+ adjustPos("S", unit)
            #adjust for right S
            if not first:
                adjustment = adjustment + adjustPos("S", unit)
            #adjust for M
            adjustment = adjustment + adjustPos("M", unit)
            #adjust for I IGNORE
            #adjustment = adjustment #- adjustPos("I", unit)
            #adjust for D
            adjustment = adjustment + adjustPos("D", unit)
            #adjust for N
            adjustment = adjustment + adjustPos("N", unit)
            unit = ""
            first = False
    return position + adjustment

#*******************************************************************************
#Function	  : adjustPos
#Description  : adjust the left most position to create right most mapping position 
#              taking into account soft clipping 
#Parameters	  : letter - what condtion is being tested for in the cigar string
#             : unit   - one number and letter unit from the cigar string
#Returned	  : int    - number that the position needs to be changed by
#*******************************************************************************

def adjustPos(letter, unit):
    condition = re.match("([0-9]+)%s" % letter, unit)
    if condition:
        return int(condition.group(1))
    else:
        return 0
    
#*******************************************************************************
#Function	  : getUMI
#Description  : get the UMI from the header line (last 8 characters in col 0) 
#Parameters	  : read - sam file read with header column
#Returned	  : UMI - the UMI for the read
#*******************************************************************************

def getUMI(read):
    read = read.strip()
    read = read.split("\t")
    read = read[0]
    UMI = read[-8:]
    return UMI

#*******************************************************************************
#Function	  : getChrom
#Description  : get the chromosome from the read (col 2) 
#Parameters	  : read - sam file read with header column
#Returned	  : chrom - the chromosome for the read
#*******************************************************************************

def getChrom(read):
    read = read.strip()
    read = read.split("\t")
    return read[2]

#*******************************************************************************
#Function	  : getStrand
#Description  : get the strandedness from bitwise flag (col 1) 
#Parameters	  : read - sam file read 
#Returned	  : bool - true if reverse strand
#*******************************************************************************

def getStrand(read):
    read = read.strip()
    read = read.split("\t")
    read = int(read[1])
    return(read & 16 == 16)

In [11]:
#UMIs = {}

try:
    UMIs
except:
    UMI_list = False
    print("Continuing with no UMI reference")
else:
    UMI_list = True
    with open("STL96.txt", "r") as fh:
        for line in fh:
            line = line.strip()
            UMIs[line] = ""

Continuing with no UMI reference


In [28]:
counter = 0
chromosome = 1
unique = {}
pairedEnd = False

file = "dataset1_100Lines.sam"

with open(file, "r") as fh:
    with open(file + "_deduped", "w") as out:
        for line in fh:
            counter = counter + 1
            if counter % 1000 == 0:
                print("On line:", counter)
            line = line.strip()
            if chromosome != getChrom(line):
                chromosome = getChrom(line)
                writeToFile(out, unique)
                unique = {}
            UMI = getUMI(line)
            if (UMI_list == True and UMI in UMIs) or UMI_list == False:
                if pairedEnd:
                    if getStrand:
                        position = adjustPosReverse(line)
                    else:
                        position = adjustPosForward(line)
                else:
                    position = adjustPosForward(line)
                identifier = UMI + str(position)
                if identifier not in unique:
                    unique[UMI, position] = line
        writeToFile(out, unique)